# 傾向スコアを用いた逆確率重み付け方(IPTW)の実装

In [1]:
# 乱数シードを固定
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.random import randint
from scipy.special import expit
from numpy.random import randn

random.seed(1234)
np.random.seed(1234)

## データの生成(chapter4_1と同じ)

In [2]:
# データ数
num_data = 200

# 年齢
x_1 = randint(15, 75, num_data) # 15から75までの一様乱数

# 性別
x_2 = randint(0, 2, num_data) # 0か1の乱数

# ノイズの発生
e_z = randn(num_data) # 平均0、標準偏差1の正規分布

# シグモイド関数に入れる部分
z_base = x_1 + (1 - x_2) * 10 - 40 + 5*e_z

# シグモイド関数を計算(CMを見る確率)
z_prob = expit(0.1 * z_base)


# テレビCMを見たかどうかの変数(0は見ていない、1は見た)
Z = np.array([])

for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p = [1 - z_prob[i], z_prob[i]])[0] # CMを見る確率をweightとして0　or　1を選ぶ
    Z = np.append(Z, Z_i)
    
# ノイズの発生
e_y = randn(num_data)

Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

df = pd.DataFrame({
    "年齢" : x_1,
    "性別" : x_2, 
    "CMを見た" : Z,
    "購入量" : Y
})

In [3]:
df.head()

,年齢,性別,CMを見た,購入量
0,62,1,1.0,88.634641
1,34,1,0.0,79.047707
2,53,0,1.0,29.331802
3,68,0,1.0,13.211139
4,27,0,1.0,64.056204


## 傾向スコアの実装

In [4]:
from sklearn.linear_model import LogisticRegression

In [5]:
# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数(目的変数)
Z = df["CMを見た"]

# 回帰の実施
reg = LogisticRegression().fit(X, Z)

In [6]:
# 回帰した結果の係数を出力
print("係数beta", reg.coef_)
print("係数alpha", reg.intercept_)

係数beta [[ 0.09025104 -0.81587482]]
係数alpha [-2.39310572]


### 結果の確認 
本来の値
- β_1 = 0.1  
- β_2 = -1
- α = -3  
---
元々仮定した式  
Y^i =-x_1 + 30x_2 + 10Z + 80 + noise  
※各係数にはシグモイド関数の係数0.1　が掛け算されることに注意

## 各人の傾向スコア

In [7]:
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])
print("-----")
print(Z[0:5]) # 5人ほどの正解

[[0.08420146 0.91579854]
 [0.53505367 0.46494633]
 [0.08392309 0.91607691]
 [0.02311324 0.97688676]
 [0.48908363 0.51091637]]
-----
0    1.0
1    0.0
2    1.0
3    1.0
4    1.0
Name: CMを見た, dtype: float64


## ATEスコアの実装

In [8]:
ATE_i = Y / Z_pre[:, 1] * Z - Y / Z_pre[:, 0] * (1 - Z )
ATE = 1 / len(Y) * ATE_i.sum()
print("推定したATE", ATE)

推定したATE 9.243658232954608


- 元々仮定した式    
Y^i =-x_1 + 30x_2 + 10Z + 80 + noise  
--- 
テレビCMを見ると購入量が10増えるモデルを仮定していた。  
ATEは約8.2となっており、テレビCMの効果(因果の大きさ)がうまく推定されている

In [9]:
ATE

9.243658232954608